In [8]:
from telethon import TelegramClient,events
import pandas as pd
import os
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
 



In [2]:
# Replace with your own values
api_id = '26559639'
api_hash = '3299ed6b2b0cbd2243470f18e1a77272'
phone = '+251984956023'

client = TelegramClient(phone, api_id, api_hash)

await client.start()

In [3]:
async def fetch_channel_messages(channel_username):
    messages = []
    async for message in client.iter_messages(channel_username, limit=100):  # Change limit as needed
        messages.append({
            'date': message.date,
            'text': message.message,
            'media': message.media
        })
    return messages

# Specify the channel username
channel_username = 'Fashiontera'  # Use the actual username or link if it's different

# Fetch messages
messages = await fetch_channel_messages(channel_username)

# Convert to DataFrame for analysis

df_messages = pd.DataFrame(messages)
df_messages.head()  # Display first few rows

,date,text,media
0,2024-09-24 14:34:31+00:00,,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
1,2024-09-24 14:34:31+00:00,,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
2,2024-09-24 14:34:31+00:00,〰️〰️〰️〰️〰️ \nCLASICO\n🔸🔸🔸🔸🔸🔸\nPrice 2200( No ...,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
3,2024-09-21 18:56:46+00:00,,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
4,2024-09-18 15:26:06+00:00,〰️〰️〰️〰️〰️〰️〰️\nPuma \nMade in Vietnam \n🔸🔸🔸🔸🔸...,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."


In [5]:
df_messages.to_csv('fashionteraa_messages.csv', index=False)

In [6]:

# Create a DataFrame to store messages
messages_list = []

@client.on(events.NewMessage(chats='Fashiontera'))
async def handler(event):
    message_data = {
        'date': event.message.date,
        'text': event.message.message,
        'media': None
    }

    # Check if the message has media (images/documents)
    if event.message.media:
        file_path = await event.message.download_media(file='documents/10thacademy/Kaim_week5/Ethiomart')  # Specify your desired directory
        message_data['media'] = file_path

    messages_list.append(message_data)

    # Save to CSV
    df_messages = pd.DataFrame(messages_list)
    df_messages.to_csv('fashiontera_messages.csv', index=False)

    print(f"Message collected: {event.message.message}")

# Start the client and run until manually stopped
await client.run_until_disconnected()

Server closed the connection: [WinError 10054] An existing connection was forcibly closed by the remote host
Server closed the connection: [WinError 10054] An existing connection was forcibly closed by the remote host
Attempt 1 at connecting failed: TimeoutError: 
Attempt 2 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 3 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 4 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 5 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 6 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 1 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 2 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 3 at connecting failed: OSError: [

ConnectionError: Connection to Telegram failed 5 time(s)

###preprocess text data

In [9]:
# Download necessary NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tesfaye\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tesfaye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# Amharic stopwords
amharic_stopwords = set(stopwords.words('amharic'))

def preprocess_text(text):
    # Normalize text: Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in amharic_stopwords]
    
    return tokens

OSError: No such file or directory: 'C:\\Users\\Tesfaye\\AppData\\Roaming\\nltk_data\\corpora\\stopwords\\amharic'

In [ ]:
# Assuming df_messages is your DataFrame containing the text data
df_messages['processed_text'] = df_messages['text'].apply(preprocess_text)

# Display the processed text
df_messages[['text', 'processed_text']].head()

In [ ]:
def normalize_amharic(text):
    # Replace common variations or errors (example)
    text = re.sub(r'ምስረዋ', 'ምስረዋ', text)  # Example of normalization
    return text

# Normalize text before tokenization
df_messages['text'] = df_messages['text'].apply(normalize_amharic)
df_messages['processed_text'] = df_messages['text'].apply(preprocess_text)

###clean and structure

In [ ]:
# Load your messages CSV file
df_messages = pd.read_csv('fashiontera_messages.csv')

# Display the first few rows
df_messages.head()

In [ ]:
def clean_and_structure_data(df):
    structured_data = []

    for index, row in df.iterrows():
        # Extract Metadata
        metadata = {
            'sender': row['sender_id'],          # Adjust based on your DataFrame structure
            'timestamp': row['date'],             # Assuming 'date' column exists
            'message_id': row['id']               # Assuming 'id' column exists
        }

        # Extract Message Content
        message_content = {
            'text': row['text'],
            'media': row['media'] if 'media' in row else None  # Handle media if present
        }

        structured_data.append({
            'metadata': metadata,
            'content': message_content
        })

    return pd.DataFrame(structured_data)

# Clean and structure the data
df_structured = clean_and_structure_data(df_messages)

# Display the structured data
df_structured.head()

###store structured data

In [ ]:
df_structured.to_csv('structured_fashiontera_messages.csv', index=False)